In [2]:
from sympy import *
import numpy as np
from scipy import linalg, sparse

## Problem Statement
stations=300
requests=50
all_variable_count = stations*requests
# Minimum Assignment from Each Station
minimumAssignment = np.random.randint(low = 1, high = 10, size = (requests, stations))
sparsity_gate=np.round(np.random.random((requests, stations))*1)
minimumAssignment=minimumAssignment*sparsity_gate
# Station Count and Request Count
station_counts=5*np.sum(minimumAssignment,axis=0)
request_counts=3*np.sum(minimumAssignment,axis=1)
# Impressions
station_impressions=np.matrix(np.random.uniform(low=10,high=20,size=stations))
request_impressions=3*(minimumAssignment*station_impressions.T).T #Target impressions for request depends on ratio of frames from stations and average station impressions


In [3]:
## Solution
# Cost function
gram_matrix=2*np.kron(station_impressions,station_impressions.T)
Q=np.kron(np.identity(requests,dtype=int),gram_matrix)
p=-2*np.kron(request_impressions,station_impressions)

# Inequality constraints - On stations and on individual non-zero variable
station_G = np.concatenate(np.apply_along_axis(lambda x: linalg.block_diag(*x),1,sparsity_gate)).T
individual_variable_G = -1*np.identity(all_variable_count,dtype=int)*sparsity_gate.reshape(1, all_variable_count)
individual_variable_G = individual_variable_G[np.where(np.sum(individual_variable_G, axis = 1) != 0)[0],]
G = np.concatenate([station_G,individual_variable_G],axis=0)

station_h = station_counts
individual_variable_h = -1*minimumAssignment[np.where(minimumAssignment != 0)]
h = np.concatenate([station_h,individual_variable_h],axis=0)

# Equality constraints - On requests and on individual zero variables
request_A = linalg.block_diag(*sparsity_gate.tolist())
individual_variable_A = np.identity(all_variable_count,dtype=int)*(1 - sparsity_gate.reshape(1, all_variable_count))
individual_variable_A = individual_variable_A[np.where(np.sum(individual_variable_A, axis = 1) != 0)[0],]
A = np.concatenate([request_A,individual_variable_A],axis=0)

request_b = request_counts
individual_variable_b = np.zeros(individual_variable_A.shape[0])
b = np.concatenate([request_b,individual_variable_b],axis=0)


In [4]:
## Solving optimization
import cvxopt as cv
sol=cv.solvers.coneqp(cv.matrix(Q), cv.matrix(p.T),
                  cv.matrix(G), cv.matrix(h), None,
                  cv.matrix(A), cv.matrix(b))

     pcost       dcost       gap    pres   dres
 0: -5.8226e+10 -5.8245e+10  2e+07  3e-15  3e-15
 1: -5.8226e+10 -5.8226e+10  2e+05  4e-16  3e-16
 2: -5.8226e+10 -5.8226e+10  2e+03  4e-16  3e-16
Optimal solution found.


In [5]:
optSol = np.round(np.matrix(sol['x']).reshape(requests, stations))
optSol

matrix([[  0.,   8.,  14., ...,  -0.,  -0.,  13.],
        [ 15.,   0.,  -0., ...,  -0.,  11.,  -0.],
        [  0.,  15.,   0., ...,  20.,  -0.,  -0.],
        ..., 
        [ 10.,  -0.,  -0., ...,  15.,  -0.,   0.],
        [  0.,  10.,  -0., ...,   0.,  -0.,   0.],
        [ 18.,   0.,  -0., ...,  -0.,  13.,  -0.]])

In [6]:
station_counts

array([ 605.,  535.,  640.,  620.,  600.,  615.,  765.,  715.,  590.,
        570.,  785.,  510.,  660.,  535.,  530.,  675.,  550.,  590.,
        610.,  675.,  490.,  825.,  625.,  625.,  710.,  560.,  650.,
        800.,  610.,  655.,  500.,  630.,  710.,  685.,  525.,  600.,
        720.,  640.,  575.,  510.,  690.,  430.,  615.,  700.,  575.,
        540.,  800.,  495.,  580.,  560.,  605.,  485.,  840.,  655.,
        610.,  620.,  585.,  635.,  450.,  650.,  635.,  705.,  460.,
        650.,  730.,  750.,  560.,  575.,  650.,  780.,  645.,  775.,
        740.,  600.,  455.,  520.,  795.,  675.,  505.,  565.,  645.,
        700.,  510.,  790.,  675.,  715.,  555.,  470.,  535.,  600.,
        525.,  505.,  565.,  835.,  605.,  635.,  705.,  450.,  575.,
        570.,  570.,  595.,  590.,  340.,  545.,  515.,  825.,  640.,
        740.,  730.,  540.,  650.,  790.,  560.,  690.,  675.,  670.,
        495.,  655.,  655.,  555.,  520.,  650.,  560.,  690.,  830.,
        715.,  665.,

In [7]:
request_counts

array([ 2412.,  2088.,  2331.,  2031.,  2175.,  1965.,  2265.,  2448.,
        2223.,  2208.,  2373.,  2283.,  2478.,  2559.,  2160.,  2343.,
        1968.,  2364.,  2280.,  2010.,  2160.,  2310.,  2160.,  2727.,
        2238.,  2178.,  2118.,  2400.,  2262.,  2295.,  2112.,  2277.,
        1785.,  1875.,  2361.,  2373.,  2253.,  2430.,  2250.,  2496.,
        2313.,  2553.,  1929.,  2160.,  2361.,  1947.,  2490.,  2322.,
        2346.,  2175.])

In [8]:
minimumAssignment

array([[ 0.,  1.,  4., ...,  0.,  0.,  6.],
       [ 6.,  0.,  0., ...,  0.,  3.,  0.],
       [ 0.,  7.,  0., ...,  8.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  2.,  0.,  0.],
       [ 0.,  2.,  0., ...,  0.,  0.,  0.],
       [ 9.,  0.,  0., ...,  0.,  6.,  0.]])

In [9]:
np.sum(optSol, axis = 0)

matrix([[ 357.,  287.,  390.,  370.,  350.,  364.,  510.,  466.,  341.,
          321.,  530.,  259.,  408.,  283.,  282.,  425.,  300.,  341.,
          358.,  423.,  242.,  571.,  369.,  374.,  460.,  312.,  398.,
          547.,  359.,  409.,  252.,  379.,  456.,  433.,  279.,  350.,
          468.,  391.,  322.,  261.,  437.,  183.,  367.,  449.,  323.,
          294.,  547.,  246.,  331.,  309.,  355.,  239.,  587.,  402.,
          361.,  369.,  336.,  384.,  204.,  399.,  381.,  451.,  211.,
          397.,  478.,  497.,  308.,  325.,  400.,  527.,  392.,  519.,
          487.,  353.,  207.,  273.,  542.,  425.,  258.,  314.,  395.,
          452.,  264.,  542.,  422.,  463.,  307.,  223.,  287.,  349.,
          275.,  258.,  316.,  580.,  353.,  382.,  452.,  205.,  329.,
          322.,  320.,  345.,  339.,   98.,  297.,  267.,  570.,  391.,
          489.,  480.,  290.,  401.,  537.,  313.,  438.,  419.,  416.,
          248.,  403.,  407.,  308.,  270.,  399.,  311.,  443.,

In [10]:
individual_variable_h

array([-1., -4., -9., ..., -9., -7., -6.])

In [11]:
np.dot(optSol, station_impressions.T)

matrix([[ 36303.22344704],
        [ 31114.6003301 ],
        [ 34769.18822937],
        [ 31313.90533081],
        [ 32612.10536904],
        [ 29675.03330519],
        [ 33660.74119019],
        [ 36999.09866088],
        [ 33504.0766914 ],
        [ 33283.95712309],
        [ 35704.20707084],
        [ 34403.73845318],
        [ 37817.94208592],
        [ 39281.804645  ],
        [ 32028.69714164],
        [ 34538.46275025],
        [ 30169.88785797],
        [ 35323.8938953 ],
        [ 34442.60780145],
        [ 29603.14372185],
        [ 32761.12963182],
        [ 34671.4630785 ],
        [ 32300.82713746],
        [ 42038.38564433],
        [ 33161.13159669],
        [ 33149.64031945],
        [ 32121.9582469 ],
        [ 37000.47182033],
        [ 34380.1384266 ],
        [ 34799.47112256],
        [ 31507.58603939],
        [ 33413.80290945],
        [ 27709.33873793],
        [ 27669.56786343],
        [ 35746.37595187],
        [ 35890.04837458],
        [ 34213.23366128],
 

In [12]:
x = np.dot(optSol, station_impressions.T) - np.dot(minimumAssignment, station_impressions.T)*3

In [13]:
np.dot(x.T, x)

matrix([[ 110911.52181558]])

In [14]:
sqrt(5.6095e+10)

236843.830403074